In [9]:
import math
import numpy as np
import matplotlib  as plt

In [10]:
class Value:
    def __init__(self,data,_child=(),_op='',):
        self.data=data
        self.prev=set(_child)
        self._back=lambda:None
        self.grad=0.0
        self._op=_op

    def __repr__(self) :
        return f"Value(data={self.data}"  
      
    def __add__(self,other):
        other =other if isinstance(other,Value) else Value(other)
        out=Value(self.data + other.data,(self,other),'+')

        def _back():
            self.grad+=1.0*out.grad
            other.grad+=1.0*out.grad
            out._back=_back 

        return out
    
    def __mul__(self,other):
        other =other if isinstance(other,Value) else Value(other)
        out=Value(self.data * other.data,(self,other),'*')

        def _back():
            self.grad+=other.grad*out.grad
            other.grad+=self.grad*out.grad
            out._back=_back
        return out
    
    def __pow__(self,other):
        assert isinstance(other,(int,float))
        out=Value(self.data**other,(self,))

        def _back():
            self.grad+=other*(self.data **(other-1))*out.grad
        out._back=_back
        return out
    
    def __rmul__(self,other):
        return self*other
    
    def __truediv__(self,other):
        return self*other**-1
    
    def tanh(self):
        x=self.data
        t=((math.exp(2*x)-1)/math.exp(2*x)+1)
        out=Value(t,(self,),'tanh')

        def _back():
            self.grad=(1-t**2)*out.grad
        out._back=_back    
        return out
    def exp(self):
        x=self.data
        out = Value(math.exp(x),(self,))
        def _back():
            self.grad += out.data*out.grad
        out._back=_back    

    


    def back(self):
    
        topo=[]
        visited =set()
        def build_topo(v):
            visited.add(v)
            for child in v.prev:
                build_topo(child)
            topo.append(v)
        build_topo(self)
        topo        
        o.grad=1.0

        for node in reversed (topo):
            node._back()


In [13]:
x1=Value(2.0)
x2=Value(0.0)
w1=Value(-3.0)
w2=Value(1.0)
b=Value(6)
x1w1=x1*w1
x2w2=x2*w2
x1w1x2w2=x1w1*x2w2
n=x1w1x2w2
o=n.tanh();

In [14]:
import torch

In [16]:
x1= torch.Tensor ([2.0]).double(); x1.requires_grad = True
x2= torch. Tensor ([0.0]).double(); x2.requires_grad = True 
w1= torch. Tensor ([-3.0]).double(); w1.requires_grad = True
w2= torch. Tensor ([1.0]).double();w2.requires_grad = True
b=torch. Tensor ([6.8813735870195432]).double(); b.requires_grad = True
n=x1*w1+x2*w2 + b
o= torch.tanh(n)

print(o.data.item())
o.backward()
print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('wl', w1.grad.item())

0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
wl 1.0000002567688737
